# MMD vs. MMDFast

In [35]:
# dependencies
import pandas as pd

In [36]:
# tool names
mmd = "mmd"
mmdfast = "mmdfast"

In [37]:
# Create a new DataFrame as a copy of the original
df = pd.read_csv('../evaluation/results.csv')

# MMD Parameter
df.loc[(df['learner'] == 0) & (df['rules'] == 'induction') & (df['coverage'] == 0.6), 'rules'] = mmd

# MMD+FR
df.loc[(df['learner'] == 3) & (df['rules'] == 'tree') & (df['coverage'] == 0.6), 'rules'] = mmdfast

# Filter rows where rules is either 'mmd' or 'mmd+fr'
df = df[df['rules'].isin([mmd, mmdfast])].reset_index()

In [38]:
df

,index,run,blackbox,model,learner,rules,coverage,seed,model_number,specificity_without,...,or_without,predicates_without,specificity_with,precision_with,recall_with,time_with,and_with,or_with,predicates_with,time_influential
0,45,1,bank,rfc,0,mmd,0.6,12345678,1,0.940,...,0,1,0.895,0.368,0.610,759.649,0,2,3,-1.000
1,46,1,bank,rfc,0,mmd,0.6,23456789,2,0.941,...,0,1,0.890,0.355,0.643,725.704,0,1,2,-1.000
2,47,1,bank,rfc,0,mmd,0.6,34567890,3,0.945,...,0,1,0.870,0.355,0.727,962.661,0,2,3,-1.000
3,48,1,bank,rfc,0,mmd,0.6,45678901,4,0.943,...,0,1,0.869,0.337,0.685,791.027,0,3,4,-1.000
4,49,1,bank,rfc,0,mmd,0.6,56789012,5,0.944,...,0,1,0.869,0.345,0.705,1006.451,0,3,4,-1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,17285,5,water,rfc,3,mmdfast,0.6,12345678,1,0.741,...,0,2,0.571,0.431,0.667,0.508,2,1,4,0.470
546,17286,5,water,rfc,3,mmdfast,0.6,23456789,2,0.351,...,0,2,0.351,0.377,0.848,0.593,1,0,2,0.569
547,17287,5,water,rfc,3,mmdfast,0.6,34567890,3,0.696,...,0,2,0.546,0.416,0.658,0.426,2,1,4,0.402
548,17288,5,water,rfc,3,mmdfast,0.6,45678901,4,0.342,...,0,2,0.342,0.403,0.821,0.490,1,0,2,0.466


In [39]:
## Calculate F1-Score

In [40]:
df['fscore'] = 2 * (df['precision_with'] * df['recall_with']) / (df['precision_with'] + df['recall_with'])

In [41]:
import pandas as pd
from scipy.stats import mannwhitneyu

def stats_for_subject(df):
    # split out the two rule‐sets
    a = df[df.rules==mmd]
    b = df[df.rules==mmdfast]

    # medians
    prec_a, rec_a, f_a = a.precision_with.median(), a.recall_with.median(), a.fscore.median()
    prec_b, rec_b, f_b = b.precision_with.median(), b.recall_with.median(), b.fscore.median()

    # Mann‐Whitney U (only if both have >1 sample)
    if len(a.fscore)>1 and len(b.fscore)>1:
        _, p = mannwhitneyu(b.fscore, a.fscore)
    else:
        p = None

    return pd.Series({
        'mmd_precision': prec_a,
        'mmd_recall':  rec_a,
        'mmd_f1':    f_a,
        f'{mmdfast}_prec': prec_b,
        f'{mmdfast}_recall':  rec_b,
        f'{mmdfast}_f1':    f_b,
        'p':        f"{p:.5f}"
    })

# 1) compute per‐subject stats
results = df.groupby('blackbox').apply(stats_for_subject).reset_index()

In [42]:
results

,blackbox,mmd_precision,mmd_recall,mmd_f1,mmdfast_prec,mmdfast_recall,mmdfast_f1,p
0,bank,0.355,0.685,0.459059,0.385,0.643,0.486292,0.00000
1,bugreport,0.227,0.736,0.346199,0.278,0.665,0.399616,0.00000
2,heartWT,0.465,0.800,0.600042,0.577,0.652,0.588275,0.81502
3,hotel,0.195,0.877,0.319058,0.203,0.850,0.328577,0.02905
4,java,0.195,0.373,0.245771,0.300,0.694,0.424390,0.00000
5,job,0.317,0.625,0.417895,0.322,0.687,0.435344,0.00756
6,php,0.136,0.627,0.225990,0.326,0.664,0.438200,0.00000
7,python,0.169,0.476,0.251482,0.344,0.624,0.450915,0.00000
8,ruby,0.144,0.614,0.233288,0.349,0.625,0.456226,0.00000
9,spam,0.379,0.624,0.471577,0.355,0.835,0.499177,0.00004
